## Import dataset

In [1]:
import pandas as pd

# Importing the dataset
dataset = pd.read_csv('795813_1365687_bundle_archive/input_bcell.csv')

dataset.head()

,parent_protein_id,protein_seq,start_position,end_position,peptide_seq,chou_fasman,emini,kolaskar_tongaonkar,parker,isoelectric_point,aromaticity,hydrophobicity,stability,target
0,A2T3T0,MDVLYSLSKTLKDARDKIVEGTLYSNVSDLIQQFNQMIITMNGNEF...,161,165,SASFT,1.016,0.703,1.018,2.22,5.810364,0.103275,-0.143829,40.273300,1
1,F0V2I4,MTIHKVAINGFGRIGRLLFRNLLSSQGVQVVAVNDVVDIKVLTHLL...,251,255,LCLKI,0.770,0.179,1.199,-3.86,6.210876,0.065476,-0.036905,24.998512,1
2,O75508,MVATCLQVVGFVTSFVGWIGVIVTTSTNDWVVTCGYTIPTCRKLDE...,145,149,AHRET,0.852,3.427,0.960,4.28,8.223938,0.091787,0.879227,27.863333,1
3,O84462,MTNSISGYQPTVTTSTSSTTSASGASGSLGASSVSTTANATVTQTA...,152,156,SNYDD,1.410,2.548,0.936,6.32,4.237976,0.044776,-0.521393,30.765373,1
4,P00918,MSHHWGYGKHNGPEHWHKDFPIAKGERQSPVDIDTHTAKYDPSLKP...,85,89,DGTYR,1.214,1.908,0.937,4.64,6.867493,0.103846,-0.578846,21.684615,1


## Check columns

In [2]:
dataset.columns

Index(['parent_protein_id', 'protein_seq', 'start_position', 'end_position',
       'peptide_seq', 'chou_fasman', 'emini', 'kolaskar_tongaonkar', 'parker',
       'isoelectric_point', 'aromaticity', 'hydrophobicity', 'stability',
       'target'],
      dtype='object')

## Search for null values

In [3]:
dataset.isnull().sum()

parent_protein_id      0
protein_seq            0
start_position         0
end_position           0
peptide_seq            0
chou_fasman            0
emini                  0
kolaskar_tongaonkar    0
parker                 0
isoelectric_point      0
aromaticity            0
hydrophobicity         0
stability              0
target                 0
dtype: int64

## Search for categorical columns

In [4]:
for col in dataset.columns:
    x = dataset[col].unique()
    if len(x) < 20:
        print(f"{col}: {x}")

target: [1 0]


## Remove irrelevant features and get input data

In [5]:
X = dataset.drop(columns=['parent_protein_id', 'protein_seq', 'peptide_seq', 'target'])
X.head()

,start_position,end_position,chou_fasman,emini,kolaskar_tongaonkar,parker,isoelectric_point,aromaticity,hydrophobicity,stability
0,161,165,1.016,0.703,1.018,2.22,5.810364,0.103275,-0.143829,40.273300
1,251,255,0.770,0.179,1.199,-3.86,6.210876,0.065476,-0.036905,24.998512
2,145,149,0.852,3.427,0.960,4.28,8.223938,0.091787,0.879227,27.863333
3,152,156,1.410,2.548,0.936,6.32,4.237976,0.044776,-0.521393,30.765373
4,85,89,1.214,1.908,0.937,4.64,6.867493,0.103846,-0.578846,21.684615


## Get output data

In [6]:
y = dataset['target']
y.head()

0    1
1    1
2    1
3    1
4    1
Name: target, dtype: int64

## Apply feature scaling

In [7]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X = sc.fit_transform(X)

X

array([[-0.3863855 , -0.40451394,  0.17066955, ...,  1.06912809,
         0.66463514, -0.20564963],
       [-0.13195355, -0.15007612, -1.80099114, ..., -0.39782531,
         0.93560006, -1.12130643],
       [-0.43161785, -0.44974733, -1.14377091, ...,  0.62331337,
         3.25724616, -0.9495729 ],
       ...,
       [-0.3072289 , -0.29708464, -1.39223222, ..., -0.15555999,
        -0.7475483 ,  0.19010766],
       [ 3.33962909,  3.34985741,  0.59545824, ..., -1.21820059,
        -0.67131514, -0.8518031 ],
       [ 3.81456874,  3.824808  , -0.26213401, ..., -1.21820059,
        -0.67131514, -0.8518031 ]])

## Split data in training and test sets

In [8]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1)

## Prepare the ANN model structure

In [9]:
# Imports
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense

# Function that build the structure of the model
def build_classifier(optimizer):
    classifier = Sequential()
    classifier.add(Dense(units = 10, kernel_initializer = 'uniform', activation = 'relu', input_dim = 10))
    classifier.add(Dense(units = 50, kernel_initializer = 'uniform', activation = 'relu'))
    classifier.add(Dense(units = 30, kernel_initializer = 'uniform', activation = 'relu'))
    classifier.add(Dense(units = 20, kernel_initializer = 'uniform', activation = 'relu'))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

## Prepare grid search to search best hyperparameters for ANN 

### AND... train all ANN

In [ ]:
# create the model
classifier = KerasClassifier(build_fn = build_classifier)

# define parameters (grid) for grid search
#   fit parameters: batch size, epochs
#   compile parameters: optimizer
parameters = {'batch_size': [10, 50],
              'epochs': [300, 500],
              'optimizer': ['adam', 'SGD']}

# define the gridsearch
    # cv = number of trainings to apply cross validation
grid_search = GridSearchCV(estimator = classifier,
                           param_grid = parameters,
                           scoring = 'accuracy',
                           cv = 10)

# fit all gridsearch combinations
grid_search = grid_search.fit(X_train, y_train)

Epoch 1/300
1166/1166 [==============================] - 1s 1ms/step - loss: 0.5662 - accuracy: 0.7278
Epoch 2/300
1166/1166 [==============================] - 1s 858us/step - loss: 0.5530 - accuracy: 0.7277
Epoch 3/300
1166/1166 [==============================] - 1s 852us/step - loss: 0.5520 - accuracy: 0.7277
Epoch 4/300
1166/1166 [==============================] - 1s 846us/step - loss: 0.5508 - accuracy: 0.7277
Epoch 5/300
1166/1166 [==============================] - 1s 855us/step - loss: 0.5493 - accuracy: 0.7267
Epoch 6/300
1166/1166 [==============================] - 1s 852us/step - loss: 0.5485 - accuracy: 0.7303
Epoch 7/300
1166/1166 [==============================] - 1s 852us/step - loss: 0.5487 - accuracy: 0.7315
Epoch 8/300
1166/1166 [==============================] - 1s 852us/step - loss: 0.5477 - accuracy: 0.7358
Epoch 9/300
1166/1166 [==============================] - 1s 847us/step - loss: 0.5452 - accuracy: 0.7363
Epoch 10/300
1166/1166 [==============================] -

Epoch 78/300
1166/1166 [==============================] - 1s 889us/step - loss: 0.4041 - accuracy: 0.8173
Epoch 79/300
1166/1166 [==============================] - 1s 906us/step - loss: 0.4036 - accuracy: 0.8162
Epoch 80/300
1166/1166 [==============================] - 1s 906us/step - loss: 0.4009 - accuracy: 0.8167
Epoch 81/300
1166/1166 [==============================] - 1s 898us/step - loss: 0.4002 - accuracy: 0.8196
Epoch 82/300
1166/1166 [==============================] - 1s 875us/step - loss: 0.4007 - accuracy: 0.8192
Epoch 83/300
1166/1166 [==============================] - 1s 858us/step - loss: 0.3979 - accuracy: 0.8190
Epoch 84/300
1166/1166 [==============================] - 1s 858us/step - loss: 0.3968 - accuracy: 0.8237
Epoch 85/300
1166/1166 [==============================] - 1s 855us/step - loss: 0.3971 - accuracy: 0.8206
Epoch 86/300
1166/1166 [==============================] - 1s 864us/step - loss: 0.3944 - accuracy: 0.8206
Epoch 87/300
1166/1166 [======================

1166/1166 [==============================] - 1s 910us/step - loss: 0.3573 - accuracy: 0.8395
Epoch 155/300
1166/1166 [==============================] - 1s 929us/step - loss: 0.3554 - accuracy: 0.8404
Epoch 156/300
1166/1166 [==============================] - 1s 927us/step - loss: 0.3562 - accuracy: 0.8386
Epoch 157/300
1166/1166 [==============================] - 1s 918us/step - loss: 0.3564 - accuracy: 0.8403
Epoch 158/300
1166/1166 [==============================] - 1s 921us/step - loss: 0.3575 - accuracy: 0.8386
Epoch 159/300
1166/1166 [==============================] - 1s 919us/step - loss: 0.3549 - accuracy: 0.8394
Epoch 160/300
1166/1166 [==============================] - 1s 921us/step - loss: 0.3552 - accuracy: 0.8404
Epoch 161/300
1166/1166 [==============================] - 1s 944us/step - loss: 0.3560 - accuracy: 0.8406
Epoch 162/300
1166/1166 [==============================] - 1s 945us/step - loss: 0.3551 - accuracy: 0.8406
Epoch 163/300
1166/1166 [==========================

1166/1166 [==============================] - 1s 931us/step - loss: 0.3354 - accuracy: 0.8520
Epoch 231/300
1166/1166 [==============================] - 1s 926us/step - loss: 0.3332 - accuracy: 0.8514
Epoch 232/300
1166/1166 [==============================] - 1s 929us/step - loss: 0.3330 - accuracy: 0.8521
Epoch 233/300
1166/1166 [==============================] - 1s 923us/step - loss: 0.3341 - accuracy: 0.8510
Epoch 234/300
1166/1166 [==============================] - 1s 926us/step - loss: 0.3347 - accuracy: 0.8525
Epoch 235/300
1166/1166 [==============================] - 1s 927us/step - loss: 0.3322 - accuracy: 0.8532
Epoch 236/300
1166/1166 [==============================] - 1s 930us/step - loss: 0.3353 - accuracy: 0.8521
Epoch 237/300
1166/1166 [==============================] - 1s 926us/step - loss: 0.3326 - accuracy: 0.8545
Epoch 238/300
1166/1166 [==============================] - 1s 926us/step - loss: 0.3339 - accuracy: 0.8526
Epoch 239/300
1166/1166 [==========================

1166/1166 [==============================] - 1s 997us/step - loss: 0.5671 - accuracy: 0.7275
Epoch 2/300
1166/1166 [==============================] - 1s 993us/step - loss: 0.5542 - accuracy: 0.7283
Epoch 3/300
1166/1166 [==============================] - 1s 958us/step - loss: 0.5508 - accuracy: 0.7327
Epoch 4/300
1166/1166 [==============================] - 1s 926us/step - loss: 0.5495 - accuracy: 0.7347
Epoch 5/300
1166/1166 [==============================] - 1s 931us/step - loss: 0.5474 - accuracy: 0.7387
Epoch 6/300
1166/1166 [==============================] - 1s 932us/step - loss: 0.5455 - accuracy: 0.7414
Epoch 7/300
1166/1166 [==============================] - 1s 925us/step - loss: 0.5389 - accuracy: 0.7480
Epoch 8/300
1166/1166 [==============================] - 1s 924us/step - loss: 0.5300 - accuracy: 0.7551
Epoch 9/300
1166/1166 [==============================] - 1s 925us/step - loss: 0.5199 - accuracy: 0.7536
Epoch 10/300
1166/1166 [==============================] - 1s 928us/

...

1166/1166 [==============================] - 1s 893us/step - loss: 0.3170 - accuracy: 0.8642
Epoch 236/300
1166/1166 [==============================] - 1s 888us/step - loss: 0.3130 - accuracy: 0.8632
Epoch 237/300
1166/1166 [==============================] - 1s 858us/step - loss: 0.3155 - accuracy: 0.8632
Epoch 238/300
1166/1166 [==============================] - 1s 907us/step - loss: 0.3154 - accuracy: 0.8619
Epoch 239/300
1166/1166 [==============================] - 1s 858us/step - loss: 0.3162 - accuracy: 0.8651
Epoch 240/300
1166/1166 [==============================] - 1s 860us/step - loss: 0.3174 - accuracy: 0.8618
Epoch 241/300
1166/1166 [==============================] - 1s 856us/step - loss: 0.3165 - accuracy: 0.8606
Epoch 242/300
1166/1166 [==============================] - 1s 853us/step - loss: 0.3129 - accuracy: 0.8640
Epoch 243/300
1166/1166 [==============================] - 1s 855us/step - loss: 0.3130 - accuracy: 0.8640
Epoch 244/300
1166/1166 [==========================

1166/1166 [==============================] - 1s 965us/step - loss: 0.5004 - accuracy: 0.7527
Epoch 12/300
1166/1166 [==============================] - 1s 934us/step - loss: 0.4938 - accuracy: 0.7548
Epoch 13/300
1166/1166 [==============================] - 1s 912us/step - loss: 0.4887 - accuracy: 0.7616
Epoch 14/300
1166/1166 [==============================] - 1s 915us/step - loss: 0.4828 - accuracy: 0.7663
Epoch 15/300
1166/1166 [==============================] - 1s 917us/step - loss: 0.4776 - accuracy: 0.7686
Epoch 16/300
1166/1166 [==============================] - 1s 916us/step - loss: 0.4715 - accuracy: 0.7716
Epoch 17/300
1166/1166 [==============================] - 1s 926us/step - loss: 0.4644 - accuracy: 0.7778
Epoch 18/300
1166/1166 [==============================] - 1s 921us/step - loss: 0.4605 - accuracy: 0.7806
Epoch 19/300
1166/1166 [==============================] - 1s 921us/step - loss: 0.4581 - accuracy: 0.7808
Epoch 20/300
1166/1166 [==============================] - 1

1166/1166 [==============================] - 1s 920us/step - loss: 0.3914 - accuracy: 0.8189
Epoch 89/300
1166/1166 [==============================] - 1s 922us/step - loss: 0.3907 - accuracy: 0.8202
Epoch 90/300
1166/1166 [==============================] - 1s 914us/step - loss: 0.3892 - accuracy: 0.8212
Epoch 91/300
1166/1166 [==============================] - 1s 922us/step - loss: 0.3898 - accuracy: 0.8186
Epoch 92/300
1166/1166 [==============================] - 1s 951us/step - loss: 0.3865 - accuracy: 0.8256
Epoch 93/300
1166/1166 [==============================] - 1s 926us/step - loss: 0.3855 - accuracy: 0.8249
Epoch 94/300
1166/1166 [==============================] - 1s 912us/step - loss: 0.3853 - accuracy: 0.8227
Epoch 95/300
1166/1166 [==============================] - 1s 915us/step - loss: 0.3838 - accuracy: 0.8231
Epoch 96/300
1166/1166 [==============================] - 1s 909us/step - loss: 0.3836 - accuracy: 0.8249
Epoch 97/300
1166/1166 [==============================] - 1

1166/1166 [==============================] - 1s 838us/step - loss: 0.3496 - accuracy: 0.8433
Epoch 165/300
1166/1166 [==============================] - 1s 870us/step - loss: 0.3436 - accuracy: 0.8486
Epoch 166/300
1166/1166 [==============================] - 1s 913us/step - loss: 0.3462 - accuracy: 0.8442
Epoch 167/300
1166/1166 [==============================] - 1s 901us/step - loss: 0.3473 - accuracy: 0.8451
Epoch 168/300
1166/1166 [==============================] - 1s 934us/step - loss: 0.3471 - accuracy: 0.8436
Epoch 169/300
1166/1166 [==============================] - 1s 932us/step - loss: 0.3461 - accuracy: 0.8432
Epoch 170/300
1166/1166 [==============================] - 1s 916us/step - loss: 0.3464 - accuracy: 0.8461
Epoch 171/300
1166/1166 [==============================] - 1s 938us/step - loss: 0.3460 - accuracy: 0.8462
Epoch 172/300
1166/1166 [==============================] - 1s 952us/step - loss: 0.3444 - accuracy: 0.8482
Epoch 173/300
1166/1166 [==========================

1166/1166 [==============================] - 1s 846us/step - loss: 0.3286 - accuracy: 0.8519
Epoch 241/300
1166/1166 [==============================] - 1s 843us/step - loss: 0.3297 - accuracy: 0.8547
Epoch 242/300
1166/1166 [==============================] - 1s 847us/step - loss: 0.3254 - accuracy: 0.8548
Epoch 243/300
1166/1166 [==============================] - 1s 846us/step - loss: 0.3281 - accuracy: 0.8562
Epoch 244/300
1166/1166 [==============================] - 1s 837us/step - loss: 0.3265 - accuracy: 0.8569
Epoch 245/300
1166/1166 [==============================] - 1s 840us/step - loss: 0.3285 - accuracy: 0.8555
Epoch 246/300
1166/1166 [==============================] - 1s 846us/step - loss: 0.3261 - accuracy: 0.8545
Epoch 247/300
1166/1166 [==============================] - 1s 836us/step - loss: 0.3283 - accuracy: 0.8577
Epoch 248/300
1166/1166 [==============================] - 1s 847us/step - loss: 0.3255 - accuracy: 0.8553
Epoch 249/300
1166/1166 [==========================

## Best parameters and accuracy

In [ ]:
# statistics
best_parameters = grid_search.best_params_
best_accuracy = grid_search.best_score_
print(str(best_parameters))
print(str(best_accuracy))

## Save the ANN

In [ ]:
# Save the model
grid_search.best_estimator_.save('epitope-prediction-v0.h5')

# Save a dictionary into a pickle file.
import pickle
pickle.dump( sc, open( "epitope-prediction-v0-sc.p", "wb" ) )

## Predict over test set and check confusion matrix

In [ ]:
# Predicting the Test set results
y_pred = grid_search.best_estimator_.predict(X_test)
y_pred = (y_pred > 0.5)

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)

import seaborn as sn
sn.heatmap(cm, cmap="Blues", annot=True,annot_kws={"size": 16})

## Get final ANN accuracy over test set

In [ ]:
print((cm[0,0] + cm[1,1]) / (cm[0,0] + cm[0,1] + cm[1,0] + cm[1,1]))